In [1]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [9]:
dim = 3
mesh_file = "/home/aben-ham/Desktop/work/stage/manapy/mesh/3D/cube.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 4573
Number of Vertices: 1140
Local domain contruction ...
---------------------------------
---------------------------------
set att: BCdirichlet
set att: BCneumann
set att: BCneumannNH
set att: cell
set att: dirichletfaces
set att: face
set att: ghost
set att: gradcellx
set att: gradcelly
set att: gradcellz
set att: gradfacex
set att: gradfacey
set att: gradfacez
set att: gradhalocellx
set att: gradhalocelly
set att: gradhalocellz
set att: halo
set att: haloghost
set att: halotosend
set att: neumannNHfaces
set att: neumannfaces
set att: node
set att: psi
set att: psihalo
---------------------------------
---------------------------------
set att: BCdirichlet
set att: BCneumann
set att: BCneumannNH
set att: cell
set att: dirichletfaces
set att: face
set att: ghost
set att: gradcellx
set att: gradcelly
set att: gradcellz
set att: gradfacex
set att: gradfacey
set att: gradfacez
set att: gradhalocellx
set att: gradhalocel

In [3]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    ne,
    domain,
    ne.domain._cells,
])

args = [
  ne.cell,
  ne.ghost,
  ne.halo,
  ne.haloghost,
  ne.domain.cells.center,
  ne.domain.cells.cellnid,
  ne.domain.cells.ghostnid,
  ne.domain.cells.haloghostnid,
  ne.domain.cells.halonid,
  ne.domain.cells.nodeid,
  ne.domain.cells.periodicnid,
  ne.domain.nodes.periodicid,
  ne.domain.faces.ghostcenter,
  ne.domain.cells.haloghostcenter,
  ne.domain.nodes.vertex,
  ne.domain.halos.centvol,
  ne.domain.cells.shift,
  ne.gradcellx,
  ne.gradcelly,
  ne.gradcellz
]



---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [4]:
from manapy.ast.functions3d import cell_gradient_3d as cpu_function
from manapy.cuda.manapy.ast.cuda_functions3d import get_kernel_cell_gradient_3d as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile cell_gradient_3d to cpu => signature=(float32[:], float32[:], float32[:], float32[:], float32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:], float32[:], float32[:])
compile kernel_cell_gradient_3d to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], float32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:], float32[:], float32[:])


In [5]:
numba_fun(*args)
%timeit numba_fun(*args)

2.52 ms ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_cell_gradient_3d.<locals>.kernel_cell_gradient_3d at 0x7e0598d125e0>) is called
1 => VarClass<float32, (4573,)>
2 => VarClass<float32, (9874,)>
3 => VarClass<float32, (0,)>
4 => VarClass<float32, (0,)>
5 => VarClass<float32, (4573, 3)>
6 => VarClass<int32, (4573, 110)>
7 => VarClass<int32, (4573, 20)>
8 => VarClass<int32, (4573, 2)>
9 => VarClass<int32, (4573, 2)>
10 => VarClass<int32, (4573, 5)>
11 => VarClass<int32, (4573, 2)>
12 => VarClass<int32, (1140, 2)>
13 => VarClass<float32, (9874, 4)>
14 => VarClass<float32, (0, 3)>
15 => VarClass<float32, (1140, 4)>
16 => VarClass<float32, (2, 2)>
17 => VarClass<float32, (4573, 3)>
18 => VarClass<float32, (4573,)>
19 => VarClass<float32, (4573,)>
20 => VarClass<float32, (4573,)>


/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


1.26 ms ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
# verify

def verify(a, decimal, plot):
  b = VarClass.to_device(a)
  if plot == True:
    plt.plot(a, label="cpu")
    plt.plot(b, label="gpu")
    plt.legend()
    plt.show()
  np.testing.assert_almost_equal(a, b, decimal=decimal)


for i, arg in enumerate(args):
  print(f"{i + 1} => test arg {arg}")
  verify(arg, decimal=2, plot=False)


1 => test arg [1. 1. 1. ... 1. 1. 1.]
2 => test arg [0. 0. 0. ... 0. 0. 0.]
3 => test arg []
4 => test arg []
5 => test arg [[0.44276604 0.43170595 0.5221834 ]
 [0.7694712  0.24118435 0.90514314]
 [0.7770873  0.31323838 0.8029697 ]
 ...
 [0.92618835 0.5116935  0.9457144 ]
 [0.03945985 0.9010808  0.98169875]
 [0.02115859 0.8817337  0.96096134]]
6 => test arg [[  19   30   33 ...   -1   -1   93]
 [   2    6   18 ...   -1   -1   85]
 [   1    4    6 ...   -1   -1   86]
 ...
 [   3    4    8 ...   -1   -1   44]
 [1192 2425 2676 ...   -1   -1   21]
 [1192 1584 2190 ...   -1   -1   22]]
7 => test arg [[  -1   -1   -1 ...   -1   -1    0]
 [1277 1279 1281 ...   -1   -1    6]
 [  -1   -1   -1 ...   -1   -1    0]
 ...
 [ 962  963  964 ...   -1   -1   16]
 [  15  222  223 ...   -1   -1   15]
 [  15   92  166 ...   -1   -1   16]]
8 => test arg [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
9 => test arg [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
10 => test arg [[ 730  804  810  904    4]
 [ 